### Упражнение 1.1

Я ознакомился с приведенными примерами.

### Упражнение 1.2

Скачайте с сайта https://freesound.org/ образец звука, включающий музыку, речь или иные звуки, имеющие четко выраженную высоту. Выделите примерно полусекундный сегмент, в котором высота постоянна. Вычислите и распечатайте спектр выбранного сегмента. Как связанны тембр звука и гармоническая структура, видимая в спепктре?

Используйте high_pass, low_pass и band_stop для фильтрации тех или иных гармоник. Затем преобразуйте спектры обратно в сигнал и прослушайте его. Как звук соотносится с изменениями, сделанными в спектре?

In [2]:
import os

if not os.path.exists('thinkdsp.py'):
    !wget https://github.com/Yuichir0/Telecom/thinkdsp.py
if not os.path.exists('334538__teddy_frost__c4.wav'):
    !wget https://github.com/Yuichir0/Telecom/334538__teddy_frost__c4.wav
from thinkdsp import read_wave

wave = read_wave('334538__teddy_frost__c4.wav')
wave.normalize()
wave.make_audio()

"wget" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.
"wget" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


ModuleNotFoundError: No module named 'thinkdsp'

Построю waveform звука

In [3]:
wave.plot()

NameError: name 'wave' is not defined

Так как выбранный звук это нота С4 на пианино, то частота на всем промежутке одинакова - выберу произвольный сегмент

In [ ]:
segment = wave.segment(start=0, duration=0.6)
segment.make_audio()

Так выглядит сегмент

In [ ]:
segment.plot()

Уменьшу отображаемый сегмент для наглядности waveform-ы

In [ ]:
segment.segment(start=0.1, duration=0.01).plot()

Разложу сегмент на спектр

In [ ]:
spectrum = segment.make_spectrum()
spectrum.plot(high=5000)

Уменьшу отображаемый диапазон

In [ ]:
spectrum = segment.make_spectrum()
spectrum.plot(high=1500)

Воспользуюсь функцией peaks для нахождения точного значения пиковых частоты

In [ ]:
spectrum.peaks()[:25]

Пиковая частота - 523.33 Гц, что равняется ноте С5, следующая по громкости частота - 1048 - это следующая октава, С6, которая также является гармонией для ноты С5.
Теперь уберу высокие частоты и сравню с оригинальным звуком

In [ ]:
spectrum.low_pass(2000)
low_filtered = spectrum.make_wave()
segment.make_audio()

In [ ]:
low_filtered.make_audio()

Звук без высоких частот звучит более низким, чем оригинальный.
Теперь воспользуюсь остальными методами для фильтрации частот - high_pass и band_stop

In [ ]:
mid_high_filtered = spectrum
mid_high_filtered.high_pass(400)
mid_high_filtered.band_stop(750, 1200)
mid_high_filtered.plot(2000)
mid_high_filtered.make_wave().make_audio()

Без ближайших гармоний звук стал более электронным и звонким

### Упражнение 1.3

Создайте сложный сигнал из объектов SinSignal и CosSignal, суммируя их. Обработайте сигнал для получения wave и прослушайте его. Вычислите Spectrum и распечатайте. Что произойдет при добавлении частот, не кратных основным?

###Решение
Попробую собрать D minor из синусов и косинусов

In [ ]:
from thinkdsp import SinSignal, CosSignal

signal = (SinSignal(freq=293.66, amp=1) +
          CosSignal(freq=329.63, amp=1) +
          SinSignal(freq=349.23, amp=1))
signal.plot()

Послушаем, что получилось

In [ ]:
dMinor = signal.make_wave(duration=1)
dMinor.apodize()
dMinor.make_audio()

Разложим волну на спектр и добавим не гармоничную частоту

In [ ]:
spectrum = dMinor.make_spectrum()
spectrum.plot(high=1000)

In [ ]:
signal += SinSignal(freq=600, amp = 1)
signal.make_wave().make_audio()

In [ ]:
Можно четко услышать случайно добавленную частоту, так как она не гармонична с остальными

### Упражнение 1.4

Напишите функцию stretch, берущую wave и коэффициент изменения. Она должна ускорять или замедлять сигнал изменением ts и framerate. Подсказка: должно получиться всего 2 строки кода.

### Решение

Ниже описана получившаяся функция

In [ ]:
def stretch(wave, rate):
    wave.ts *= rate
    wave.framerate /= rate

Воспользуюсь ей на исходном примере, ускорив его в 2 раза

In [ ]:
wave = read_wave('334538__teddy_frost__c4.wav')
wave.normalize()
wave.make_audio()
stretch(wave, 0.5)
wave.plot()

In [ ]:
wave.make_audio()

Теперь замедлю его в 4 раза

In [ ]:
stretch(wave, 4)
wave.plot()

In [ ]:
wave.make_audio()